# 📢 Disclaimer

This notebook contains material copied verbatim from the [LlamaIndex documentation](https://www.llamaindex.ai/)  
and was created with the assistance of ChatGPT.  

It is intended for educational purposes only.  
All copyrights and credits belong to the LlamaIndex team and their respective authors.



# Streaming events

Workflows can be complex -- they are designed to handle complex, branching, concurrent logic -- which means they can take time to fully execute. To provide your user with a good experience, you may want to provide an indication of progress by streaming events as they occur. Workflows have built-in support for this on the `Context` object.

In [2]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)
import asyncio
from llama_index.llms.openai import OpenAI
from llama_index.utils.workflow import draw_all_possible_flows


---

## 📚 What does `import asyncio` mean?

```python
import asyncio
```

- It **imports** the **`asyncio` module** — Python’s **built-in library** for **asynchronous programming**.
- `asyncio` helps you **run multiple tasks at the same time** (without blocking), **wait** for results, **pause and resume** operations efficiently.

✅ You use `asyncio` when you want to **handle many operations** (like API calls, downloads, I/O tasks) **without slowing down** your program.

---

## 🧩 Why do we need `asyncio`?

Because when you use `async` and `await` in Python,  
you need something to **manage** and **run** those `async` functions.

That "something" is `asyncio`.

✅ `asyncio` runs your **coroutines** — async functions that you can pause and resume.

---

## 🎯 Example usage

Here’s a **basic full example**:

```python
import asyncio

async def say_hello():
    print("Hello...")
    await asyncio.sleep(1)
    print("...World!")

async def main():
    await say_hello()

if __name__ == "__main__":
    asyncio.run(main())
```

- `async def` defines async functions.
- `await asyncio.sleep(1)` **pauses** for 1 second **without blocking** everything.
- `asyncio.run(main())` **runs** the whole async program.

✅ The whole thing is managed by `asyncio`.

---

## 🏡 Real-World Analogy

Imagine you are baking cookies:

- **Normal code (no asyncio):**  
  ➔ You bake one cookie at a time. You wait... you do nothing else.

- **Asyncio code:**  
  ➔ You bake one tray, while waiting you make coffee, answer emails, and when the oven beeps, you continue baking!

✅ **Multitasking efficiently.**

---

## 📜 Markdown Summary you can paste into your Notebook

```markdown
## 🛠️ What is `import asyncio`?

`asyncio` is Python's built-in library for **asynchronous programming**.

When we use `async` and `await`, we need `asyncio` to manage and run those async functions.

## Simple example:
```python
import asyncio

async def say_hello():
    print("Hello...")
    await asyncio.sleep(1)
    print("...World!")

async def main():
    await say_hello()

if __name__ == "__main__":
    asyncio.run(main())
```

✅ `asyncio.run(main())` starts the async program.
✅ `await` pauses tasks without blocking everything else.

---



## Load API Keys for OpenAI

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

Let's set up some events for a simple three-step workflow, plus an event to handle streaming our progress as we go

In [6]:
class FirstEvent(Event):
    first_output: str


class SecondEvent(Event):
    second_output: str
    response: str


class ProgressEvent(Event):
    msg: str

In [9]:
class MyWorkflow(Workflow):
    @step
    async def step_one(self, ctx: Context, ev: StartEvent) -> FirstEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step one is happening"))
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ctx: Context, ev: FirstEvent) -> SecondEvent:
        llm = OpenAI(model="gpt-4o-mini")
        generator = await llm.astream_complete(
            "Please give me the first 3 paragraphs of Moby Dick, a book in the public domain."
        )
        async for response in generator:
            # Allow the workflow to stream this piece of response
            ctx.write_event_to_stream(ProgressEvent(msg=response.delta))
        return SecondEvent(
            second_output="Second step complete, full response attached",
            response=str(response),
        )

    @step
    async def step_three(self, ctx: Context, ev: SecondEvent) -> StopEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step three is happening"))
        return StopEvent(result="Workflow complete.")

In [21]:
w = MyWorkflow(timeout=300, verbose=True)
handler = w.run(first_input="Start the workflow.")

async for ev in handler.stream_events():
        if isinstance(ev, ProgressEvent):
            print(ev.msg)

final_result = await handler
print("Final result", final_result)

draw_all_possible_flows(MyWorkflow, filename="streaming_workflow.html")

Running step step_one
Step step_one produced event FirstEvent
Running step step_two
Step one is happening

Sure
!
 Here
 are
 the
 first
 three
 paragraphs
 of
 "
M
oby
 Dick
"
 by
 Herman
 Mel
ville
:


---


Call
 me
 Ish
mael
.
 Some
 years
 ago
—
never
 mind
 how
 long
 precisely
—I
 had
 little
 or
 no
 money
 in
 my
 purse
,
 and
 nothing
 particular
 to
 interest
 me
 on
 shore
.
 So
,
 whenever
 I
 find
 myself
 growing
 grim
 about
 the
 mouth
;
 whenever
 it
 is
 a
 damp
,
 dr
izzly
 November
 in
 my
 soul
;
 whenever
 I
 find
 myself
 invol
unt
arily
 pa
using
 before
 coffin
 warehouses
,
 and
 bringing
 up
 the
 rear
 of
 every
 funeral
 I
 meet
;
 and
 especially
 whenever
 my
 hy
pos
 get
 such
 an
 upper
 hand
 of
 me
,
 that
 it
 requires
 a
 strong
 moral
 principle
 to
 prevent
 me
 from
 deliberately
 stepping
 into
 the
 street
,
 and
 method
ically
 knocking
 people
’s
 hats
 off
—
then
,
 I
 account
 it
 high
 time
 to
 get
 to
 sea
 as
 soon
 as
 I
 can
.
 This


gio: file:///home/tshepiso/workspace/coldblooded-agents/LlamaIndex-Tutorials/worflows/streaming_workflow.html: Failed to find default application for content type ‘text/html’


`run` runs the workflow in the background, while stream_events will provide any event that gets written to the stream. It stops when the stream delivers a `StopEvent`, after which you can get the final result of the workflow as you normally would.

In [23]:
handler

<WorkflowHandler finished result='Workflow complete.'>